# Step 5 - Analysis of bicycle network results
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and 04_connect_clusters and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency
* overlap with existing networks

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-11-24

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Analyze existing infrastructure

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Analyzing existing infrastructure.")
    
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    filename = placeid + "_existing.csv"
    if not os.path.isfile(PATH["results"] + placeid + "/" + filename):
        empty_metrics = {
                         "length":0,
                         "length_lcc":0,
                         "coverage": 0,
                         "directness": 0,
                         "directness_lcc": 0,
                         "poi_coverage": 0,
                         "components": 0,
                         "efficiency_global": 0,
                         "efficiency_local": 0
                        }
        output_place = {}
        for networktype in networktypes:
            output_place[networktype] = copy.deepcopy(empty_metrics)

        # Analyze all networks     
        Gs = {}
        for networktype in networktypes:
            Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
            Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
        
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_railwaystation_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

        covs = {}
        for networktype in networktypes:
            if debug: print(placeid + ": Analyzing results: " + networktype)
            metrics, cov, _ = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, buffer_overlap, numnodepairs, debug)
            for key, val in metrics.items():
                output_place[networktype][key] = val
            covs[networktype] = cov
        # Save the covers
        write_result(covs, "pickle", placeid, "", "", "existing_covers.pickle")
        
        # Write to CSV
        write_result(output_place, "dictnested", placeid, "", "", "existing.csv", empty_metrics)

## Analyze POI based results

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    Gexisting = {}
    for networktype in ["biketrack", "bikeable"]:
        Gexisting[networktype] = ig_to_shapely(csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified"))
        
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
         
    # Calculate
    # output contains lists for all the prune_quantile values of the corresponding results
    output, covs, output_carminusbike, covs_carminusbike, output_carconstrictedbike = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, buffer_overlap, numnodepairs, debug, True, Gexisting)
    output_MST, cov_MST, _ = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, buffer_overlap, numnodepairs, debug, True, ig.Graph(), Polygon(), Polygon(), False, Gexisting)
        
    # Save the covers
    write_result(covs, "pickle", placeid, poi_source, prune_measure, "_covers.pickle")
    write_result(covs_carminusbike, "pickle", placeid, poi_source, prune_measure, "_covers_carminusbike.pickle")
    write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_cover_mst.pickle")
        
    # Write to CSV
    write_result(output, "dict", placeid, poi_source, prune_measure, ".csv")
    write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
    write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")
    write_result(output_MST, "dict", placeid, poi_source, prune_measure, "_mst.csv")
    
#     filename = placeid + '_poi_' + poi_source + "_mst.csv"
#     with open(PATH["results"] + placeid + "/" + filename, 'w') as f:
#         w = csv.writer(f)
#         w.writerow(output_MST.keys())
#         w.writerow(output_MST.values())

In [ ]:
# To do: Move this cell to 06 and make a nice cover plot
if debug:
#     pp.pprint(output)
    for prune_quantile, cov in covs.items():
        fig = plt.figure(figsize=(4, 3)) # create figure object with a (width,height)
        axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
        if isinstance(cov, shapely.geometry.multipolygon.MultiPolygon):
            for pol in cov: # cov is generally a MultiPolygon, so we iterate through its Polygons
                y, x = pol.exterior.coords.xy
                patch1 = matplotlib.patches.Polygon(np.stack((np.asarray(y), -1*np.asarray(x)), axis=-1))
                axes.add_patch(patch1)
        elif isinstance(cov, shapely.geometry.polygon.Polygon) and not cov.is_empty:
            y, x = cov.exterior.coords.xy
            patch1 = matplotlib.patches.Polygon(np.stack((np.asarray(y), -1*np.asarray(x)), axis=-1))
            axes.add_patch(patch1)
        axes.set_title(str(prune_quantile))
        axes.set_ylim([51.355, 51.40])
        axes.set_xlim([-2.40, -2.312])
        axes.plot()

## Analyze cluster based results

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")
for placeid, placeinfo in cities.items():
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    
    # Load results
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".pickle"
    resultfile = open(PATH["results"] + placeid + "/" + filename,'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    if debug: pp.pprint(res)
        
        
    # output contains lists for all the prune_quantile values of the corresponding results
    output = {"length":[],
              "length_lcc":[],
              "coverage": [],
              "directness": [],
              "directness_lcc": [],
              "poi_coverage": [],
              "components": [],
              "efficiency_global": [],
              "efficiency_local": []
             }

    covs = {}
    for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
        if debug: print(prune_quantile, len(GT.vs))
        metrics, cov = calculate_metrics(GT, GT_abstract, G_carall, nnids, output, buffer_walk, numnodepairs)
        for key in output.keys():
            output[key].append(metrics[key])
        covs[prune_quantile] = cov
    # Save the covers
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + "_covers.pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'wb') as f:
        pickle.dump(covs, f)
        
    # Write to CSV
    filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".csv"
    with open(PATH["results"] + placeid + "/" + filename, 'w') as f:
        w = csv.writer(f)
        w.writerow(output.keys())
        w.writerows(zip(*output.values()))

In [ ]:
Audio(sound_file, autoplay=True)